In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

from model.inference import * 
from utils.access_data import *
from utils.constants import *

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

In [ ]:
IMG_DIR = DATA_DIR + 'GALE_CRATER/cartOrder/cartorder/'
image_file = IMG_DIR + 'layered_img_sec_100_150.pickle'
wavelengths_file = IMG_DIR + 'layered_wavelengths.pickle'

# Normalize spectra across RELAB, USGS, and CRISM per each CRISM image
# (since different CRISM images have different wavelengths)
record_reduced_spectra(wavelengths_file)

image = get_CRISM_data(image_file, wavelengths_file, CRISM_match=True)
print("CRISM image size " + str(image.shape))

## Testing
plot spectra of random pixels from image. frt0002037a_07_if165l_trr3_CAT

In [ ]:
# load "l" image, reset all NULLs to 0 
CRISM_DATA_PATH = DATA_DIR + 'GALE_CRATER_TEST_2/'
CRISM_IMG = CRISM_DATA_PATH + 'frt0000c0ef_07_if165l_trr3_CAT.img'
spy_image = envi.open(file=CRISM_IMG + '.hdr')


image_arr = spy_image[:,:,:]
img= np.where(image_arr[:,:,:] == 65535, 0, image_arr) 
# S_IMG_WAVELENGTHS = CRISM_DATA_PATH + 'l_pixel_x_201_y_200.csv'
wavelengths = get_CRISM_wavelengths(CRISM_DATA_PATH + 'pixel_x_262_y_136.csv')



print(len(wavelengths))
print(img.shape)

bands = (300, 200, 50)
from spectral import imshow
imshow(data=img, bands=bands)

In [ ]:
#  height = 450
#  width = 640

In [ ]:
from spectral import imshow

def plot_cutout_spectra(img, wavelengths, sec_width, sec_height, xstart, ystart, bands):
    """
    Visualize subsection of image and corresponding spectra to see variance
    :param sec_width: number of columns to include
    :param sec_height: number of rows to include
    :param xstart: column to start at (where 0 is left most column)
    :param ystart: row to start at (where 0 is top row)
    """
    fig, ax = plt.subplots(2, 1, constrained_layout=True,  dpi=300 ) # figsize=(4, 2), dpi=DPI
    
    height, width, num_wavelengths = img.shape
    
    avg_spectra = np.zeros(num_wavelengths)
    num_pixels = sec_width*sec_height
     
    for i in range(sec_height):
        for j in range(sec_width): 
                            # img [ height, width ]
            pixel_spectra = img[i+ystart,j+xstart]
            ax[0].plot(wavelengths, pixel_spectra, linewidth=0.5)
            avg_spectra += pixel_spectra 
            
             
    avg_spectra = avg_spectra/num_pixels
    ax[0].plot(wavelengths, avg_spectra, linewidth=1.0, color='red')
    
    ax[0].set_xlabel("Wavelength")
    ax[0].set_ylabel("Reflectance")
    ax[0].set_title("Spectra")
    ax[0].set_ylim((0, 1))

    for i in range(sec_height):
        for j in range(sec_width): 
            pixel_spectra = img[i+ystart,j+xstart] 
            ax[1].plot(wavelengths, pixel_spectra-avg_spectra)
    ax[1].set_title("Normalied Spectra (avg subtracted)")
    ax[1].set_xlabel("Wavelength")
    ax[1].set_ylabel("reflectance - avg")
    ax[1].set_ylim((0,.5))
    
    plt.show()
    
    
     
    view = imshow(data=img[xstart:(xstart+sec_height),ystart:(ystart+sec_width),:], bands=bands)
    return avg_spectra/num_pixels

    

bands = (300, 200, 50)
avg_spectra=plot_cutout_spectra(img=img,
                    wavelengths=wavelengths,
                    sec_width = 600,
                    sec_height = 400,
                    xstart = 20,
                    ystart = 20,
                    bands=bands)


## PT sampler testing

In [ ]:
from emcee import PTSampler

# mu1 = [1, 1], mu2 = [-1, -1]
mu1 = np.ones(2)
mu2 = -np.ones(2)

# Width of 0.1 in each dimension
sigma1inv = np.diag([100.0, 100.0])
sigma2inv = np.diag([100.0, 100.0])

def logl(x):
    dx1 = x - mu1
    dx2 = x - mu2

    return np.logaddexp(-np.dot(dx1, np.dot(sigma1inv, dx1))/2.0,
                        -np.dot(dx2, np.dot(sigma2inv, dx2))/2.0)

# Use a flat prior
def logp(x):
    return 0.0

In [ ]:
ntemps = 4
nwalkers = 10
ndim = 2

num_burnin_iterations = 100

sampler=PTSampler(ntemps, nwalkers, ndim, logl, logp)
p0 = np.random.uniform(low=-1.0, high=1.0, size=(ntemps, nwalkers, ndim))
for p, lnprob, lnlike in sampler.sample(p0, iterations=num_burnin_iterations):
    pass
sampler.reset()

In [ ]:
# At each iteration, this generator for PTSampler yields

# p, the current position of the walkers.
# lnprob the current posterior values for the walkers.
# lnlike the current likelihood values for the walkers.

In [ ]:
num_iterations = 100

for p, lnprob, lnlike in sampler.sample(p0=p, lnprob0=lnprob,
                                           lnlike0=lnlike,
                                           iterations=num_iterations, 
                                        thin=10):
    pass

In [ ]:
assert sampler.chain.shape == (ntemps, nwalkers, 20, ndim)

In [ ]:
assert sampler.chain.shape == (ntemps, nwalkers, 1000, ndim)

# Chain has shape (ntemps, nwalkers, nsteps, ndim)
# Zero temperature mean:
mu0 = np.mean(np.mean(sampler.chain[0,...], axis=0), axis=0)

# Longest autocorrelation length (over any temperature)
max_acl = np.max(sampler.acor)

# etc

## Infer point

Try to infer single endmember.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

from model.inference import *
from model.hapke_model import get_USGS_r_mixed_hapke_estimate
from preprocessing.generate_USGS_data import generate_image
from utils.plotting import *
from utils.access_data import *
from utils.constants import *

def get_rmse(a, b): 
    return np.sqrt(np.mean((a - b)**2))

def print_error(m_actual, D_actual, m_est, D_est): 
    m_rmse = str(round(get_rmse(m_actual, m_est), 2))
    D_rmse = str(round(get_rmse(D_actual, D_est), 2))
    print(str(np.array(D_actual)) + ", "  + str(m_est) + ", " + str(m_rmse) + ", " + str(D_est) + ", " + str(D_rmse))
    return m_rmse, D_rmse

plot_endmembers(CRISM_match=False)

In [ ]:
def test_inference(m_sample, D_sample, name):
    true_m = convert_arr_to_dict(m_sample)
    true_D = convert_arr_to_dict(D_sample)
    r_actual = get_USGS_r_mixed_hapke_estimate(m=true_m, D=true_D)
    est_m, est_D = infer_datapoint(d=r_actual,
                                   iterations=NUM_ITERATIONS, 
                                   C=10, 
                                   V=GRAIN_SIZE_COVARIANCE)
    m_rmse, D_rmse=print_error(m_sample, D_sample, est_m, est_D)
    wavelengths = get_USGS_wavelengths()
    r_est = get_USGS_r_mixed_hapke_estimate(convert_arr_to_dict(est_m),  convert_arr_to_dict(est_D))
    fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(FIG_WIDTH, FIG_HEIGHT), dpi=DPI)
    ax.plot(wavelengths, r_est, label = "Estimated", color = "orange")
    ax.plot(wavelengths, r_actual, label = "Actual", color="blue")
    ax.set_xlabel("Wavelength")
    ax.set_ylabel("Reflectance")
    ax.legend()
    ax.set_title("RMSE M: " + str(m_rmse) + ", D: " + str(D_rmse))
    plt.ylim((0, 1)) 
    plt.savefig("../output/tests/" + str(name) + ".pdf") 

In [ ]:

# ["augite", "enstatite", "labradorite",  "olivine (Fo51)"]
m_sample = [0.6, 0.05,.05,0.4]
D_sample = [200, 300, 100, 200]
NAME = "mixed_7"
GRAIN_SIZE_COVARIANCE = 50
NUM_ITERATIONS = 15
test_inference(m_sample, D_sample, NAME)

## Synthetic Image Generation
Create synthetic image resembling some realistic geology to have single, useful test image.

In [ ]:
#Create 2D Numpy Matrix based on circles
from matplotlib.lines import Line2D
import matplotlib
def points_in_circle_np(radius, x0=0, y0=0):
    """
    Get X, Y coords for given circle
    """
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    # x, y = np.where((np.hypot((x_-x0)[:,np.newaxis], y_-y0)<= radius)) # alternative implementation
    for x, y in zip(x_[x], y_[y]):
        yield x, y

        

img_width = 24
img_height = 24
color_img = np.zeros((img_width,img_height,3))
test_img = np.zeros((img_width,img_height,1))

center_X = int(img_height/2)
center_Y = int(img_width/2)
outer_circle = points_in_circle_np(9, x0=center_X, y0=center_Y)
medium_circle =  points_in_circle_np(6, x0=center_X, y0=center_Y)
inner_circle =  points_in_circle_np(3, x0=center_X+1, y0=center_Y+1)
lower_ring = points_in_circle_np(11, x0=center_X, y0=center_Y-1)

circles = [lower_ring, 
           outer_circle, 
           medium_circle,
           inner_circle]
LR_C = [255, 102, 255]
O_C = [102, 255, 204]
M_C = [51, 51, 255]
I_C = [51, 102, 255]

RGB_MAP = {0: LR_C,
          1: O_C,
          2: M_C,
          3: I_C}
DEFAULT_COLOR =  [153, 51, 51]

for row_index, row in enumerate(color_img):
    for col_index, col in enumerate(row):
        color_img[row_index,col_index] = DEFAULT_COLOR
        test_img[row_index,col_index] = 0
            
for i, circle in enumerate(circles):
    for point in circle:
        x=point[0]
        y=point[1] 
        if i != 0:
            color_img[x,y] = RGB_MAP[i]
            test_img[x,y] = i + 1
        else:
            # only do lower, ring so threshold x 
            if x >= int(img_height*0.6):
                color_img[x,y] = RGB_MAP[i]
                test_img[x,y] = i + 1
# Plot image
figure, ax = plt.subplots(1)
color_img=np.array(color_img,np.int32)
            
plt.imshow(color_img)

a = Line2D([0], [0], color='#%02x%02x%02x' % tuple(DEFAULT_COLOR), lw=4) 
clines = [a] 
for c in list(RGB_MAP.values()):  
    hex_color='#%02x%02x%02x' % tuple(c)
    a = Line2D([0], [0], color=hex_color, lw=4) 
    clines.append(a)

ax.legend(clines, [ "Background", "Lower ring", "Outer", "Inner", "Peak"],loc=(1.1,0.5))

plt.savefig(PREPROCESSED_DATA + "SYNTHETIC/visual.pdf", bbox_inches='tight')

In [ ]:
# ["augite", "enstatite", "labradorite",  "olivine (Fo51)"]
testing_regions = ["b", "l", "o", "i", "p"]
m_type_map = { 0:"b", 1:"l", 2 :"o", 3 :"i", 4:"p"}
region_ms = {"b": [0.3, 0.2,.1,0.4],
                "l": [0.7, 0, 0.3,0],
                "o": [0, 0.6,0.4,0],
                "i": [0.8, 0.2, 0, 0],
                "p": [0.3, 0, 0, 0.7]}
region_Ds = {"b": [200, 300, 100, 200],
                "l": [200, 300, 100, 200],
                "o": [200, 300, 100, 200],
                "i": [200, 300, 100, 200],
                "p": [200, 300, 100, 200]}
region_Rs = {}

# Mix each endmember; and then just add a bit of noise when creating image
for ttype in testing_regions: 
    m_map = {}
    D_map = {}
    for index, endmember in enumerate(USGS_PURE_ENDMEMBERS):
        m_map[endmember] = region_ms[ttype][index]
        D_map[endmember] = region_Ds[ttype][index]
    r = get_USGS_r_mixed_hapke_estimate(m_map, D_map)
    region_Rs[ttype] = r
    
with open(R_DIR + "../wavelengths.pickle", 'rb') as handle:
    wavelengths = pickle.load(handle)

    
r_image = np.zeros((img_width,img_height,len(wavelengths)))
m_image = np.zeros((img_width,img_height,len(USGS_PURE_ENDMEMBERS)))
D_image = np.zeros((img_width,img_height,len(USGS_PURE_ENDMEMBERS)))

# Fill test image with synthetically mixed spectra
for row_index, row in enumerate(test_img):
    for col_index, col in enumerate(row):
        ttype_index = test_img[row_index, col_index]
        ttype = testing_regions[int(ttype_index)]
        cur_R = region_Rs[ttype]
        # Add noise here
#         noise = np.random.normal(loc=0,
#                                      scale=noise_scale,
#                                      size=len(wavelengths))
        r_image[row_index, col_index] = cur_R #  + noise
        m_image[row_index, col_index] = region_ms[ttype]
        D_image[row_index, col_index] = region_Ds[ttype]
        

with open(PREPROCESSED_DATA + "SYNTHETIC/r_img.pickle", 'wb') as f:
    pickle.dump(r_image, f)
with open(PREPROCESSED_DATA + "SYNTHETIC/m_actual.pickle", 'wb') as f: 
    pickle.dump(m_image, f)
with open(PREPROCESSED_DATA + "SYNTHETIC/D_actual.pickle", 'wb') as f:
    pickle.dump(D_image, f)

## Synthetic image testing

Determine why there are columns of pixels with same inference

In [ ]:
import numpy as np
import math
import os
import pickle

from model.models import *
from testing.run_inference import *
from preprocessing.generate_USGS_data import generate_image
from utils.plotting import *
from utils.constants import *

with open(PREPROCESSED_DATA + "SYNTHETIC/m_actual.pickle", 'rb') as F:
    m_actual = pickle.load(F)
with open(PREPROCESSED_DATA + "SYNTHETIC/D_actual.pickle", 'rb') as F:
    D_actual = pickle.load(F)
with open(PREPROCESSED_DATA + "SYNTHETIC/r_img.pickle", 'rb') as F:
    R_image = pickle.load(F)

row_min = 1
row_max = 5
col_min = 1
col_max = 5

m_actual = m_actual[row_min:row_max,col_min:col_max,:]
D_actual = D_actual[row_min:row_max,col_min:col_max,:]
R_image = R_image[row_min:row_max,col_min:col_max,:]


In [ ]:


m_est, D_est = ind_model(iterations=200,
                         image=R_image,
                         C=10,
                         V=50)

EXP_NAME = "TEST_SYNTHETIC2"
if not os.path.exists('../output/' + EXP_NAME):
    os.makedirs('../output/' + EXP_NAME)
record_output(m_actual, D_actual, m_est, D_est, "ind/", EXP_NAME)